# Advent of Code Day 23
#### Problem in full can be found here: https://adventofcode.com/2023/day/23

In [111]:
# Given an input file that acts as a maze, find the longest path given from a starting location to a
# end location
# Part 1: has slopes '>' '<' 'v' '^' where it forces one direction
# Part 2: remove the slopes

# Get the input in a map
file = open("inputday23.txt")
map = []
for line in file:
    line = line.strip()
    map.append(list(line))
file.close()

# Since the input is too big and it takes too long to backtrack through everything,
# the solution will first put everything in a weighted graph to drastically reduce
# the number of visited locations and only visit those where multiple paths can be taken
g = makeGraph(map, {}, (0, 1), (140, 139), set(), True)
print(findMaxDistance(g, (0, 1), 0, 0))
g2 = makeGraph(map, {}, (0, 1), (140, 139), set(), False)
print(findMaxDistance(g2, (0, 1), 0, 0))

2358
6182


In [106]:
# This function recrusively creates a graph by given a starting node, collects the distance
# of 1 way paths until it reaches a location where more than 1 path is possible or no paths
# are possible (as once the end is reached, no paths are available)
def makeGraph(map, graph, start, end, visited, slopes):
    # Initialize a queue and the distance
    q = deque([start])
    distance = 0

    # While the queue is not empty
    while q:
        # Get the current location
        current = q.pop()
        # Add the current location to the set of visited locations
        visited.add(current)

        # Get the possible paths
        possiblePaths = getPaths(map, current, visited, slopes)
        # If there is one possible path only, increase the distance and add the path to the queue
        if len(possiblePaths) == 1:
            q.append(possiblePaths[0])
            distance += 1
        # If there is 0 or more than 1 path, update the graph and if there is more than 1 path,
        # recursively call the function again with the new starting location
        else:
            graph[start] = (possiblePaths, distance)
            for path in possiblePaths:
                graph = makeGraph(map, graph, path, end, visited, slopes)

    # Return the graph
    return graph

In [109]:
# This function returns a list of possible paths given the current location
def getPaths(map, current, visited, slopes):
    # Initialize the variables
    x, y = current[0], current[1]
    symbol = map[x][y]
    paths = []

    # If the direction of the path can be any direction
    if symbol == '.' or not slopes:
        for direction in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            if 0 <= x + direction[0] < len(map) and 0 <= y + direction[1] < len(map):
                newX, newY = x + direction[0], y + direction[1]
                if map[newX][newY] != '#' and (newX, newY) not in visited:
                    paths.append((newX, newY))
    # If the direction of the path has to be a specific direction
    elif symbol == 'v': 
        paths.append((x + 1, y))
    elif symbol == '^': 
        paths.append((x - 1, y))
    elif symbol == '>': 
        paths.append((x, y + 1))
    elif symbol == '<': 
        paths.append((x, y - 1))

    # Return the paths
    return paths

In [62]:
# This function recursively finds the maximum distance by looping through all paths and finding
# the distance from the start to the end
def findMaxDistance(g, index, distance, maxDistance):
    # If the length is 0, it means this is the end
    if len(g[index][0]) == 0:
        # Add up the final distance to finish off the path
        distance += g[index][1]
        # Take the max of the current distance and max distance
        maxDistance = max(distance, maxDistance)
        # Return the max distance
        return maxDistance

    # If it is not the end (the list is not []) then recursively call the function
    # with updated variables for all the paths
    for path in g[index][0]:
        maxDistance = findMaxDistance(g, path, distance+g[index][1], maxDistance)

    # Return the max distance
    return maxDistance